In [ ]:
import pandas as pd
pd.set_option('display.float_format', '{:.1f}'.format)

In [ ]:
files = ['triton-tensorRT/bench-triton-tensorRT-llm.csv',
'triton-tensorRT-quantized/bench-triton-tensorRT-llm-quantized.csv',
'triton-vllm-awq/bench-triton-vllm-awq.csv',
'triton-tensorRT-quantized-awq/bench-triton-tensorRT-llm-quantized-awq.csv']

In [ ]:
df = pd.concat([pd.read_csv(f) for f in files])
result = df.groupby('note')[['tok_count', 'time']].mean().reset_index()

In [ ]:
result['Tok/Sec'] = result.tok_count / result.time
result = result.rename(columns={'tok_count': 'Avg Tok Count', 
               'time': 'Avg Time'})
result = result.sort_values(by='Tok/Sec', ascending=True).reset_index(drop=True)

In [ ]:
def backend(note):
    if note == 'triton-tensorRT-llm':
        return 'TensorRT-LLM'
    if note == 'triton-tensorRT-llm-quantized':
        return 'TensorRT-LLM'
    if note == 'triton-vllm-awq':
        return 'vLLM'
    if note == 'triton-tensorRT-llm-quantized-awq':
        return 'TensorRT-LLM'

def quant(note):
    if note == 'triton-tensorRT-llm':
        return 'None'
    if note == 'triton-tensorRT-llm-quantized':
        return 'INT8 weight-only quantization'
    if note == 'triton-vllm-awq':
        return 'AWQ 4bit quantization'
    if note == 'triton-tensorRT-llm-quantized-awq':
        return 'AWQ 4bit quantization'

result['Backend'] = result['note'].apply(backend)
result['Quantization']= result['note'].apply(quant)
result[['Backend', 'Quantization', 'Avg Tok Count', 'Avg Time', 'Tok/Sec']]

,Backend,Quantization,Avg Tok Count,Avg Time,Tok/Sec
0,TensorRT-LLM,None,225.6,3.3,69.0
1,TensorRT-LLM,INT8 weight-only quantization,225.6,1.8,125.3
2,vLLM,AWQ 4bit quantization,223.6,1.6,142.2
3,TensorRT-LLM,AWQ 4bit quantization,225.6,1.1,205.5
